In [ ]:
# Minimal causal VAE (CEVAE)
# Binary treatment 𝑇
# Continuous outcome 𝑌
# Observed covariates 𝑋
# Latent confounder 𝑍

In [ ]:
# Model the joint distribution as: p(Z) p(X∣Z)p(T∣Z)p(Y∣T, Z)
# and use the variational inference: q(Z∣X, T, Y)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal, Bernoulli


In [3]:
class Encoder(nn.Module):
    def __init__(self, x_dim, z_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(x_dim + 2, hidden_dim),  # X + T + Y
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.mu = nn.Linear(hidden_dim, z_dim)
        self.logvar = nn.Linear(hidden_dim, z_dim)

    def forward(self, x, t, y):
        h = self.net(torch.cat([x, t, y], dim=1))
        return self.mu(h), self.logvar(h)


In [4]:
class XDecoder(nn.Module):
    def __init__(self, z_dim, x_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, x_dim)
        )

    def forward(self, z):
        return self.net(z)


In [6]:
class TDecoder(nn.Module):
    def __init__(self, z_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, z):
        return torch.sigmoid(self.net(z))


In [7]:
class YDecoder(nn.Module):
    def __init__(self, z_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim + 1, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, z, t):
        return self.net(torch.cat([z, t], dim=1))


In [8]:
class CausalVAE(nn.Module):
    def __init__(self, x_dim, z_dim):
        super().__init__()
        self.encoder = Encoder(x_dim, z_dim)
        self.x_decoder = XDecoder(z_dim, x_dim)
        self.t_decoder = TDecoder(z_dim)
        self.y_decoder = YDecoder(z_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, t, y):
        mu_z, logvar_z = self.encoder(x, t, y)
        z = self.reparameterize(mu_z, logvar_z)

        x_hat = self.x_decoder(z)
        t_hat = self.t_decoder(z)
        y_hat = self.y_decoder(z, t)

        return x_hat, t_hat, y_hat, mu_z, logvar_z


In [9]:
def elbo_loss(x, t, y, x_hat, t_hat, y_hat, mu_z, logvar_z):
    # Reconstruction
    x_loss = F.mse_loss(x_hat, x, reduction="sum")
    t_loss = F.binary_cross_entropy(t_hat, t, reduction="sum")
    y_loss = F.mse_loss(y_hat, y, reduction="sum")

    # KL divergence
    kl = -0.5 * torch.sum(1 + logvar_z - mu_z.pow(2) - logvar_z.exp())

    return x_loss + t_loss + y_loss + kl


In [10]:
model = CausalVAE(x_dim=10, z_dim=5)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for x, t, y in dataloader:
    optimizer.zero_grad()

    x_hat, t_hat, y_hat, mu_z, logvar_z = model(x, t, y)
    loss = elbo_loss(x, t, y, x_hat, t_hat, y_hat, mu_z, logvar_z)

    loss.backward()
    optimizer.step()


NameError: name 'dataloader' is not defined

In [ ]:
def counterfactual_y(model, x, t_obs, y_obs, t_cf):
    mu_z, logvar_z = model.encoder(x, t_obs, y_obs)
    z = model.reparameterize(mu_z, logvar_z)
    return model.y_decoder(z, t_cf)


In [ ]:
y0 = counterfactual_y(model, x, t, y, t_cf=torch.zeros_like(t))
y1 = counterfactual_y(model, x, t, y, t_cf=torch.ones_like(t))
ite = y1 - y0
